In [13]:
import pandas as pd

preproc_file = "processed_data.csv"

df = pd.read_csv(preproc_file, sep=';', engine='python')

therapy_applied = df.query("TherapyApply == 1")
therapy_not_applied = df.query("TherapyApply == 0")



In [14]:
therapy_applied.head()

,CaseID,Gender,Age,Start,End,Outcome,Vacin,TherapyApply,Результат_F,Результат_D,Severity,Meds
0,376917,ж,65.0,2021-05-08,2021-07-14,Выписан,Нет,1,135.9,423.90,Крайне тяжелое,0
1,390158,м,73.0,2021-05-30,2021-07-14,Выписан,Нет,1,1024.0,578.65,Тяжелое,0
2,390185,ж,57.0,2021-05-31,2021-07-14,Умер,Нет,1,606.0,349.60,Крайне тяжелое,0
3,390093,ж,66.0,2021-06-01,2021-07-06,Умер,Нет,1,586.9,486.70,Крайне тяжелое,0
4,390165,м,56.0,2021-06-01,2021-07-12,Выписан,Спутник V,1,1013.0,417.30,Крайне тяжелое,0


In [15]:
therapy_not_applied.head()

,CaseID,Gender,Age,Start,End,Outcome,Vacin,TherapyApply,Результат_F,Результат_D,Severity,Meds
2451,390819,ж,79.0,2021-05-07,2021-06-30,Выписан,#Н/Д,0,565.25,853.6,Тяжелое,0
2452,390409,ж,49.0,2021-05-11,2021-06-25,Выписан,#Н/Д,0,387.90,285.9,Тяжелое,0
2453,390794,ж,65.0,2021-05-20,2021-06-29,Выписан,#Н/Д,0,606.00,349.6,Тяжелое,0
2454,390448,м,44.0,2021-05-21,2021-06-25,Выписан,#Н/Д,0,1082.00,320.6,Среднетяжелое,0
2455,390760,ж,69.0,2021-05-27,2021-06-25,Выписан,#Н/Д,0,586.90,486.7,Тяжелое,0


In [16]:
therapy_applied_men = therapy_applied.query("Gender == 'м'")
therapy_applied_women = therapy_applied.query("Gender == 'ж'")

therapy_not_applied_men = therapy_not_applied.query("Gender == 'м'")
therapy_not_applied_women = therapy_not_applied.query("Gender == 'ж'")



In [18]:
therapy_applied_men.head()

,CaseID,Gender,Age,Start,End,Outcome,Vacin,TherapyApply,Результат_F,Результат_D,Severity,Meds
1,390158,м,73.0,2021-05-30,2021-07-14,Выписан,Нет,1,1024.0,578.65,Тяжелое,0
4,390165,м,56.0,2021-06-01,2021-07-12,Выписан,Спутник V,1,1013.0,417.30,Крайне тяжелое,0
5,390089,м,63.0,2021-06-02,2021-06-25,Умер,Нет,1,1013.0,417.30,Тяжелое,0
6,390142,м,56.0,2021-06-03,2021-07-13,Выписан,Нет,1,1013.0,417.30,Тяжелое,0
8,390164,м,42.0,2021-06-04,2021-06-30,Выписан,Нет,1,1082.0,320.60,Крайне тяжелое,0


In [28]:

print(f"Men applied therapy {therapy_applied_men.shape}")
print(f"Women applied therapy {therapy_applied_women.shape}")
print(f"Men not applied therapy {therapy_not_applied_men.shape}")
print(f"Women not applied therapy {therapy_not_applied_women.shape}")


Men applied therapy (1059, 12)
Women applied therapy (1392, 12)
Men not applied therapy (2343, 12)
Women not applied therapy (3704, 12)


In [48]:
import numpy as np
def generate_pairs(applied, not_applied):
    applied = applied.sort_values("Age")
    not_applied = not_applied.sort_values("Age")
    pairs = []
    used = np.zeros(not_applied.index.max(), dtype=bool)
    """
    mask = not_applied[
        (not_applied['Gender'] == applied['Gender']) & 
        (abs(not_applied['Age'] - applied['Age']) <= 3) &
        (abs(not_applied['Результат_F'] - applied['Результат_F') / not_applied['Результат_F' <= 0.1) &
        (abs(not_applied['Результат_D'] - applied['Результат_D') / not_applied['Результат_D' <= 0.1) &
        ((not_applied['Severity'] == "Крайне тяжелое") | (not_applied['Severity'] == "Тяжелое")) & 
        (applied['Severity'] == "Среднетяжелое")
        ]
    """
    for index, first in applied.iterrows():
        filtered = not_applied[
                    (not_applied['Gender'] == first['Gender']) & 
                    (abs(not_applied['Age'] - first['Age']) <= 3) &
                    (abs(not_applied['Результат_F'] - first['Результат_F']) / not_applied['Результат_F'] <= 0.1) &
                    (abs(not_applied['Результат_D'] - first['Результат_D']) / not_applied['Результат_D'] <= 0.1) &
                    ((not_applied['Severity'] == "Крайне тяжелое") | (not_applied['Severity'] == "Тяжелое")) & 
                    (first['Severity'] == "Среднетяжелое")
        ]
        for sIndex, second in filtered.iterrows():
            if not used[sIndex]:
                pairs.append([first, second])
                used[sIndex] = True
    return pairs
pairs_men = generate_pairs(therapy_applied_men, therapy_not_applied_men)
pairs_women = generate_pairs(therapy_applied_women, therapy_not_applied_women)